In [199]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [200]:
#load the dataset 
data=pd.read_csv(r"Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [201]:
#firest 3 features are not much imp for churn prediction 
#Row number , customerID, Surname , so will drop these features as these are irrelevent 
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [202]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [203]:
#we can see in dataset Geography and Gender are catogorical features will perform label encoding on those
labelencoder_Gender=LabelEncoder()    # LabelEncoder is class , need to create object of it 
labelencoder_Gender.fit(data['Gender'])  #labelencoder is useful in case of binary classification
data['Gender']=labelencoder_Gender.transform(data['Gender']) 
#for 0 and 1 conversion but if we use label encoder for multiclass classification then it will convert ito
#1 ,2, 3, 4 depend on classes but in ANN we need values in beteen 0 to 1 only. 
#that is why it is ok to use label encoder for gender but for geography will use OHE

data.head()                      

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [204]:
trained_classes = labelencoder_Gender.classes_
trained_classes

array(['Female', 'Male'], dtype=object)

In [207]:
#apply OHE on Geography
from sklearn.preprocessing import OneHotEncoder
OHE=OneHotEncoder(sparse_output=False)
encoder_geo=OHE.fit_transform(data[['Geography']])
encoder_geo

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [208]:
OHE.get_feature_names_out()

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [209]:
geo_encoded_df=pd.DataFrame(encoder_geo,columns=OHE.get_feature_names_out())
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [210]:
# combine One hot encoded columns with origional dataframe
#before this just delete original geography column 
new_data=data.drop(['Geography'],axis=1)
new_data=pd.concat([new_data,geo_encoded_df],axis=1)

In [211]:
#save the lable enconer and one hot encoder in pickle file so that later on we can we this 
with open('labelencoder_Gender.pkl','wb') as file:
    pickle.dump(labelencoder_Gender,file)

with open('OHE_geo.pkl','wb') as file:
    pickle.dump(OHE,file)

In [131]:
new_data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [132]:
#Divide data into dependent and indepent features 
X=new_data.drop(['Exited'],axis=1)
Y=new_data['Exited']

In [133]:
#split data into training and testing set
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

In [134]:
#feature scaling 
scalar=StandardScaler()
x_train=scalar.fit_transform(x_train)
x_test=scalar.transform(x_test)

In [136]:
pd.DataFrame(x_train)

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.356500,0.913248,-0.655786,0.345680,-1.218471,0.808436,0.649203,0.974817,1.367670,1.001501,-0.579467,-0.576388
1,-0.203898,0.913248,0.294938,-0.348369,0.696838,0.808436,0.649203,0.974817,1.661254,-0.998501,1.725723,-0.576388
2,-0.961472,0.913248,-1.416365,-0.695393,0.618629,-0.916688,0.649203,-1.025834,-0.252807,-0.998501,-0.579467,1.734942
3,-0.940717,-1.094993,-1.131148,1.386753,0.953212,-0.916688,0.649203,-1.025834,0.915393,1.001501,-0.579467,-0.576388
4,-1.397337,0.913248,1.625953,1.386753,1.057449,-0.916688,-1.540351,-1.025834,-1.059600,1.001501,-0.579467,-0.576388
...,...,...,...,...,...,...,...,...,...,...,...,...
7995,1.207474,0.913248,1.435808,1.039728,-0.102301,-0.916688,0.649203,0.974817,-0.539860,1.001501,-0.579467,-0.576388
7996,0.314989,-1.094993,1.816097,-1.389442,-1.218471,-0.916688,0.649203,0.974817,-1.733882,1.001501,-0.579467,-0.576388
7997,0.865009,-1.094993,-0.085351,-1.389442,-1.218471,2.533560,-1.540351,-1.025834,-0.142765,1.001501,-0.579467,-0.576388
7998,0.159323,0.913248,0.390011,1.039728,1.827259,-0.916688,0.649203,-1.025834,-0.050826,1.001501,-0.579467,-0.576388


In [137]:
#save this scaling in pickle file 
with open('scalar.pkl','wb') as file:
    pickle.dump(scalar,file)

In [138]:
#Build ANN model



In [139]:
#import required libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [149]:
print(x_train.shape)
print(x_train.shape[0])
print(x_train.shape[1])   #below we need how many inputs we are connecting that is why we need this

(8000, 12)
8000
12


In [159]:
#build ANN model
model=Sequential([
    Dense(64,activation='relu',input_shape=(x_train.shape[1],)),  #need to specify input only for first hidden layer
    Dense(32,activation='relu'),  #HL2
    Dense(1,activation='sigmoid')   #output layer
])

In [160]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_11 (Dense)                │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [162]:
# #compile the model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy']) 
#  #here adam optimizer here use default learning rate 

In [163]:
# to custmize learning rate for any optimizer can use 
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)

model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy']) 

In [170]:
# setup tensorboard before model training to capture the logs and visualize

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [171]:
#setup earlystoping 
#early stopping is required to stop epochs , if we specify 100 epochs and our model got well trained in 50 epochs
# it means loss function reduced to sufficent level and after 50 it will reduce very negligibly then earlybird 
# help to stop trainig 

Early_Stopping_callback=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)
#for monitoring loss will use validation loss, we have multiple loss but now monitories based on validation loss
#patience - take patience for at least 5 epochs , if no imrovement after 5 epochs then stop training
#restore_best_weights - at which epoch you get best weight reload that and get that with early stopping 

In [172]:
#training the model 
history=model.fit(
    x_train,y_train,validation_data=(x_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,Early_Stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8634 - loss: 0.3333 - val_accuracy: 0.8535 - val_loss: 0.3635
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8703 - loss: 0.3212 - val_accuracy: 0.8540 - val_loss: 0.3454
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8690 - loss: 0.3249 - val_accuracy: 0.8555 - val_loss: 0.3510
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8675 - loss: 0.3187 - val_accuracy: 0.8570 - val_loss: 0.3434
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8699 - loss: 0.3201 - val_accuracy: 0.8575 - val_loss: 0.3439
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8695 - loss: 0.3139 - val_accuracy: 0.8570 - val_loss: 0.3633
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8674 - loss: 0.3236 - val_accuracy: 0.8520 - val_loss: 0.3568
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8754 - loss: 0.3059 - val_accu

In [173]:
model.save('modebl.h5')

In [174]:
#load tensorboad extension 
%load_ext tensorboard

In [176]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 31940), started 0:02:00 ago. (Use '!kill 31940' to kill it.)